### Project 1 - Solution

In [31]:
template = {
    'user_id': int,
    'name': {
        'first': str,
        'last': str
    },
    'bio': {
        'dob': {
            'year': int,
            'month': int,
            'day': int
        },
        'birthplace': {
            'country': str,
            'city': str
        }
    }
}

So, a JSON document such as this would match the template:

In [32]:
john = {
    'user_id': 100,
    'name': {
        'first': 'John',
        'last': 'Cleese'
    },
    'bio': {
        'dob': {
            'year': 1939,
            'month': 11,
            'day': 27
        },
        'birthplace': {
            'country': 'United Kingdom',
            'city': 'Weston-super-Mare'
        }
    }
}

But this one would **not** match the template (missing key):

In [33]:
eric = {
    'user_id': 101,
    'name': {
        'first': 'Eric',
        'last': 'Idle'
    },
    'bio': {
        'dob': {
            'year': 1943,
            'month': 3,
            'day': 29
        },
        'birthplace': {
            'country': 'United Kingdom'
        }
    }
}

And neither would this one (wrong data type):

In [34]:
michael = {
    'user_id': 102,
    'name': {
        'first': 'Michael',
        'last': 'Palin'
    },
    'bio': {
        'dob': {
            'year': 1943,
            'month': 'May',
            'day': 5
        },
        'birthplace': {
            'country': 'United Kingdom',
            'city': 'Sheffield'
        }
    }
}

That should return this:
* `validate(john, template) --> True, ''`
* `validate(eric, template) --> False, 'mismatched keys: bio.birthplace.city'`
* `validate(michael, template) --> False, 'bad type: bio.dob.month'`

In [ ]:
def validate(data, template):
    # implement
    # and return True/False
    # in the case of False, return a string describing
    # the first error encountered
    # in the case of True, string can be empty
    return state, error

In [18]:
def match_keys(data, valid, path):
    data_keys = data.keys()
    valid_keys = valid.keys()

    extra_keys = data_keys - valid_keys
    missing_keys = valid_keys - data_keys

    if missing_keys or extra_keys:
        missing_msg = ('missing keys: ' +
                       ', '.join({path + '.' + str(key)
                                 for key in missing_keys})
                                 ) if missing_keys else ''
        extras_msg = ('extra keys: ' +
                     ', '.join({path + '.' + str(key)
                               for key in extra_keys})
                               ) if extra_keys else ''
        return False, ' '.join((missing_msg, extras_msg))
    else:
        return True, None

In [19]:
t = {'a': int, 'b': int, 'c': int, 'd': {}}
d = {'a': 'wrong type', 'b': 100, 'c': 200, 'd': {'wrong', 'type'}}

In [20]:
is_ok, err_msg = match_keys(d, t, 'some.path')
print(is_ok, err_msg)

True None


In [21]:
d = {'a': None, 'b': None, 'c': None}
is_ok, err_msg = match_keys(d, t, 'some.path')
print(is_ok, err_msg)

False missing keys: some.path.d 


In [22]:
d = {'a': None, 'b': None, 'c': None, 'd': None, 'e': None}
is_ok, err_msg = match_keys(d, t, 'some.path')
print(is_ok, err_msg)

False  extra keys: some.path.e


In [23]:
d = {'a': None, 'b': None, 'c': None, 'e': None}
is_ok, err_msg = match_keys(d, t, 'some.path')
print(is_ok, err_msg)

False missing keys: some.path.d extra keys: some.path.e


In [24]:
d = {'a': None, 'b': None, 'e': None, 'f': None}
is_ok, err_msg = match_keys(d, t, 'some.path')
print(is_ok, err_msg)

False missing keys: some.path.d, some.path.c extra keys: some.path.e, some.path.f


In [25]:
def match_types(data, template, path):
    for key, value in template.items():
        if isinstance(value, dict):
            template_type = dict
        else:
            template_type = value
        data_value = data.get(key, object())
        if not isinstance(data_value, template_type):
            err_msg = ('incorrect type: ' + path + '.' + key +
                       ' -> expected ' + template_type.__name__ +
                       ', found ' + type(data_value).__name__)
            return False, err_msg
    return True, None

In [26]:
t = {'a': int, 'b': str, 'c': {'d': int}}
d = {'a': 100, 'b': 'test', 'c': {'some': 'value'}}

In [27]:
match_types(d, t, 'some.path')

(True, None)

In [28]:
d = {'a': 100, 'b': 'test', 'c': 'unexpected'}
match_types(d, t, 'some.path')

(False, 'incorrect type: some.path.c -> expected dict, found str')

In [29]:
d = {'a': 100, 'b': 200, 'c': 'unexpected'}
match_types(d, t, 'some.path')

(False, 'incorrect type: some.path.b -> expected str, found int')

In [30]:
def recurse_validate(data, template, path):
    is_ok, err_msg = match_keys(data, template, path)
    if not is_ok:
        return False, err_msg

    is_ok, err_msg = match_types(data, template, path)
    if not is_ok:
        return False, err_msg

    dictionary_type_keys = {key for key, value in template.items()
                            if isinstance(value, dict)}

    for key in dictionary_type_keys:
        sub_path = path + '.' + str(key)
        sub_template = template[key]
        sub_data = data[key]
        is_ok, err_msg = recurse_validate(sub_data, sub_template, sub_path)
        if not is_ok:
            return False, err_msg

    return True, None


In [35]:
is_ok, err_msg = recurse_validate(john, template, 'root')

In [36]:
print(is_ok, err_msg)

True None


In [37]:
recurse_validate(eric, template, 'root')

(False, 'missing keys: root.bio.birthplace.city ')

In [38]:
recurse_validate(michael, template, 'root')

(False, 'incorrect type: root.bio.dob.month -> expected int, found str')

In [42]:
def validate(data, tempalte):
    return recurse_validate(data, template, '')

In [43]:
persons = ((john, 'John'), (eric, 'Eric'), (michael, 'Michael'))

In [44]:
for person, name in persons:
    is_ok, err_msg = validate(person, template)
    print(f'{name}: valid={is_ok}: {err_msg}')

John: valid=True: None
Eric: valid=False: missing keys: .bio.birthplace.city 
Michael: valid=False: incorrect type: .bio.dob.month -> expected int, found str


In [46]:
class SchemaError(Exception):
    pass

In [47]:
def validate(data, template):
    is_ok, err_msg = recurse_validate(data, template, '')
    if not is_ok:
        raise SchemaError(err_msg)

In [48]:
for person, name in persons:
    validate(person, template)

SchemaError: missing keys: .bio.birthplace.city 

In [49]:
validate(john, template)

In [50]:
validate(eric, template)

SchemaError: missing keys: .bio.birthplace.city 

In [52]:

for person, name in persons:
    try:
        validate(person, template)
    except SchemaError as ex:
        print('Validataion failed', str(ex))

Validataion failed missing keys: .bio.birthplace.city 
Validataion failed incorrect type: .bio.dob.month -> expected int, found str


In [54]:
class SchemaError(Exception):
    pass

class SchemaKeyMismatch(SchemaError):
    pass

class SchemaTypeMismatch(SchemaError, TypeError):
    pass

In [55]:
def match_keys(data, valid, path):
    data_keys = data.keys()
    valid_keys = valid.keys()

    extra_keys = data_keys - valid_keys
    missing_keys = valid_keys - data_keys

    if missing_keys or extra_keys:
        missing_msg = ('missing keys: ' +
                       ', '.join({path + '.' + str(key)
                                 for key in missing_keys})
                                 ) if missing_keys else ''
        extras_msg = ('extra keys: ' +
                     ', '.join({path + '.' + str(key)
                               for key in extra_keys})
                               ) if extra_keys else ''
        raise SchemaKeyMismatch(' '.join((missing_msg, extras_msg)))

In [56]:
def match_types(data, template, path):
    for key, value in template.items():
        if isinstance(value, dict):
            template_type = dict
        else:
            template_type = value
        data_value = data.get(key, object())
        if not isinstance(data_value, template_type):
            err_msg = ('incorrect type: ' + path + '.' + key +
                       ' -> expected ' + template_type.__name__ +
                       ', found ' + type(data_value).__name__)
            raise SchemaTypeMismatch(err_msg)

In [61]:
def recurse_validate(data, template, path):

    match_keys(data, template, path)
    match_types(data, template, path)

    dictionary_type_keys = {key for key, value in template.items()
                            if isinstance(value, dict)}

    for key in dictionary_type_keys:
        sub_path = path + '.' + str(key)
        sub_template = template[key]
        sub_data = data[key]
        recurse_validate(sub_data, sub_template, sub_path)

In [62]:
def validate(data, template):
    recurse_validate(data, template, '')

In [63]:
validate(john, template)

In [64]:
validate(eric, template)

SchemaKeyMismatch: missing keys: .bio.birthplace.city 

In [65]:
validate(michael, template)

SchemaTypeMismatch: incorrect type: .bio.dob.month -> expected int, found str

In [66]:
try:
    validate(john, template)
except SchemaError as ex:
    print(ex)

In [67]:
try:
    validate(eric, template)
except SchemaError as ex:
    print(ex)

missing keys: .bio.birthplace.city 


In [71]:
try:
    validate(michael, template)
except SchemaTypeMismatch as ex:
    print(ex)

incorrect type: .bio.dob.month -> expected int, found str


In [74]:
try:
    validate(eric, template)
except SchemaKeyMismatch as ex:
    print('handling a key mismatch exception', ex)
except SchemaTypeMismatch as ex:
    print('handling a type mismatch exception', ex)
except SchemaError as ex:
    print('handling some general schema exceptions', ex)

handling a key mismatch exception missing keys: .bio.birthplace.city 


In [75]:
try:
    validate(michael, template)
except SchemaKeyMismatch as ex:
    print('handling a key mismatch exception', ex)
except SchemaTypeMismatch as ex:
    print('handling a type mismatch exception', ex)
except SchemaError as ex:
    print('handling some general schema exceptions', ex)

handling a type mismatch exception incorrect type: .bio.dob.month -> expected int, found str


In [76]:
try:
    validate(michael, template)
except SchemaKeyMismatch as ex:
    print('handling a key mismatch exception', ex)

except SchemaError as ex:
    print('handling some general schema exceptions', ex)
except TypeError as ex:
    print('handling a general type exception', ex)

handling some general schema exceptions incorrect type: .bio.dob.month -> expected int, found str


In [77]:
try:
    validate(michael, template)
except SchemaError as ex:
    print('handling some general schema exceptions', ex)
except TypeError as ex:
    print('handling a general type exception', ex)

handling some general schema exceptions incorrect type: .bio.dob.month -> expected int, found str
